In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm
import time

In [2]:
import rotation_matrix as rm

In [3]:
def connect_robot(ip="192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip)  # IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [4]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [6]:
import numpy as np

In [7]:
import numpy as np

data = np.loadtxt(
    "../pouring_simulation/output/MediumBottle/Medium_50_200_66/TCP.txt",
    delimiter=",",
    skiprows=1,
)

In [8]:
data.shape[0]

4675

In [9]:
# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254
data[:, 2] = data[:, 2] + 0.25307274

In [10]:
data

array([[-2.33210100e-05,  3.78973080e-05,  2.53572740e-01],
       [-4.66117940e-05,  7.58068080e-05,  2.54072740e-01],
       [-6.98903860e-05,  1.13728500e-04,  2.54572740e-01],
       ...,
       [ 2.33331258e-05, -3.78853700e-05,  2.53072740e-01],
       [ 2.33331258e-05, -3.78853700e-05,  2.53072740e-01],
       [ 2.33331258e-05, -3.78853700e-05,  2.53072740e-01]])

In [11]:
start_pos = rtde_r.getActualTCPPose()

start_pos

[-0.05816210105224679,
 0.5216683472820339,
 0.16989501136970128,
 -1.5486796346850868,
 0.2657775035471559,
 0.26434027391587783]

In [12]:
start_pos = [
    -0.0555077809037947,
    0.5267063264795802,
    0.1640237148198558,
    -1.5588760353504112,
    0.03869269986900248,
    -0.023812772591892194,
]

In [13]:
rtde_c.moveL(start_pos, 0.1, 0.1)

True

In [14]:
init_q = rtde_r.getActualQ()
init_q

[-4.016320530568258,
 -1.6673509083189906,
 -2.4532134532928467,
 4.130081339473389,
 -0.8258827368365687,
 -3.1330071131335657]

In [15]:
init_q[5] += 0.25307274
rtde_c.moveJ(init_q, 0.1, 0.1)

True

In [16]:
start_pos_flask2 = rtde_r.getActualTCPPose()

In [17]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    # positions.append([data[i,0], data[i,1], 0.0, 0.0, 0.0, data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups
    positions.append(
        [data[i, 0], -data[i, 1], 0.0, 0.0, 0.0, data[i, 2]]
    )  # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [18]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(
        rm.PoseTrans(start_pos, positions[i])
    )  # transform from base to tool coordinates(positions[i]))

In [19]:
positions_converted

[[-0.05553073170005485,
  0.526706804676422,
  0.16406183447016734,
  -1.5440616788534982,
  0.2358930711676873,
  0.17610541780734074],
 [-0.05555365218287278,
  0.5267072815259287,
  0.16409996601736287,
  -1.5440144958902229,
  0.2362808908686231,
  0.1764993109494699],
 [-0.05557656037111536,
  0.5267077577456512,
  0.16413810963669642,
  -1.5439672420975719,
  0.2366687057952205,
  0.17689320212484633],
 [-0.05559944433915042,
  0.526708232863605,
  0.16417626495818358,
  -1.5439199174746945,
  0.23705651594305985,
  0.17728709132903697],
 [-0.055622309919375425,
  0.5267087071062081,
  0.16421443254653917,
  -1.543872522020738,
  0.23744432130772078,
  0.1776809785576086],
 [-0.05564516332018142,
  0.5267091808667205,
  0.16425259976244194,
  -1.5438250557348474,
  0.23783212188478314,
  0.17807486380612786],
 [-0.05566799264176054,
  0.5267096533908913,
  0.16429079087358628,
  -1.5437775186161657,
  0.23821991766982603,
  0.17846874707016105],
 [-0.05569079722380779,
  0.526710

In [20]:
# get first 5 x and z positions
positions = positions_converted[0:15]

# only get x and z positions
positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions) - 1):
    distances.append(np.linalg.norm(np.subtract(positions[i + 1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i] * 60)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

[4.4490037358183674e-05, 4.4494053360194835e-05, 4.449162337809105e-05, 4.449269108413219e-05, 4.448611354308685e-05, 4.4494256867622026e-05, 4.449270226442049e-05, 4.44951665014802e-05, 4.4496637102409487e-05, 4.448739866639912e-05, 4.449539383766049e-05, 4.448748975206186e-05, 4.450045577343141e-05, 4.448730183293589e-05]
[0.0026694022414910203, 0.00266964320161169, 0.002669497402685463, 0.002669561465047931, 0.002669166812585211, 0.0026696554120573214, 0.0026695621358652295, 0.002669709990088812, 0.0026697982261445694, 0.002669243919983947, 0.0026697236302596294, 0.0026692493851237114, 0.0026700273464058846, 0.0026692381099761536]
Speed in m/s:  0.0026695342342376123


In [21]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif (
        positions_converted[i]
        == positions_converted[i - 1]
        == positions_converted[i - 2]
        == positions_converted[i - 3]
        == positions_converted[i - 4]
    ):
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

2305
8


In [22]:
# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

In [23]:
# print length of both lists
print(len(positions_converted1))
print(len(positions_converted2))

2305
2370


In [38]:
velocity = avg_speed  # 0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.002
blend_3 = 0.0
# path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
# path.append(path_pose1)
for i in range(len(positions_converted1) - 1):
    path.append(
        [
            positions_converted1[i][0],
            positions_converted1[i][1],
            positions_converted1[i][2],
            positions_converted1[i][3],
            positions_converted1[i][4],
            positions_converted1[i][5],
            velocity,
            acceleration,
            blend_i,
        ]
    )

path.append(
    [
        positions_converted1[-1][0],
        positions_converted1[-1][1],
        positions_converted1[-1][2],
        positions_converted1[-1][3],
        positions_converted1[-1][4],
        positions_converted1[-1][5],
        velocity,
        acceleration,
        0,
    ]
)
rtde_c.moveL(path)

time.sleep(count / 60)

path_2 = []
for i in range(len(positions_converted2) - 1):
    path_2.append(
        [
            positions_converted2[i][0],
            positions_converted2[i][1],
            positions_converted2[i][2],
            positions_converted2[i][3],
            positions_converted2[i][4],
            positions_converted2[i][5],
            velocity,
            acceleration,
            blend_i,
        ]
    )

path_2.append(
    [
        positions_converted2[-1][0],
        positions_converted2[-1][1],
        positions_converted2[-1][2],
        positions_converted2[-1][3],
        positions_converted2[-1][4],
        positions_converted2[-1][5],
        velocity,
        acceleration,
        blend_3,
    ]
)

rtde_c.moveL(path_2)

rtde_c.stopScript()

In [71]:
velocity = avg_speed  # 0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.003
blend_3 = 0.0
# path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
# path.append(path_pose1)
for i in range(data.shape[0] - 1):
    path.append(
        [
            positions_converted[i][0],
            positions_converted[i][1],
            positions_converted[i][2],
            positions_converted[i][3],
            positions_converted[i][4],
            positions_converted[i][5],
            velocity,
            acceleration,
            blend_i,
        ]
    )


# path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
# path.append(path_pose3)

rtde_c.moveL(path)

rtde_c.stopScript()

AttributeError: 'rtde_control.RTDEControlInterface' object has no attribute 'startScript'